# Bayes Opt Hacking

In [1]:
%load_ext autoreload
%autoreload 2
from belief_active_plot import BeliefActivePlot

Using backend: pytorch


In [87]:
# Running functions
args = {
    'net': 'VariationalGaussianProcessRegressor',
    'architecture': ['GraphSAGE', '32', 'activation', 'tanh',
                     'GraphSAGE', '32', 'activation', 'tanh',
                     'GraphSAGE', '32', 'activation', 'tanh'],
    'optimizer': 'Adam', # Adam
    'lr': 1e-4,
    'data': 'moonshot_sorted',
    'strategy': 'sequential',
    'acquisition': 'ExpectedImprovement',
    'num_thompson_samples': 1000,
    'num_samples': 1000,
    'q': 96,
    'device': 'cuda:0',
    'num_trials': 20, # 9
    'num_rounds': 80, # 80
    'num_epochs': 200, # 20
    'index_provided': True,
    'early_stopping': False,
    'beliefs': ['ThompsonSampling'],
    'annealing': 1.0,
    'n_inducing_points': 100,
    'sample_frac': 1.0
}

from types import SimpleNamespace
args = SimpleNamespace(**args)

plot = BeliefActivePlot(
    # net config
    net=args.net,
    architecture=args.architecture,
    n_inducing_points=args.n_inducing_points,
    annealing=args.annealing,

    # optimizer config
    optimizer_type=args.optimizer,
    lr=args.lr,

    # experiment config
    data=args.data,
    sample_frac=args.sample_frac,
    acquisition=args.acquisition,
    num_samples=args.num_samples,
    num_thompson_samples=args.num_thompson_samples,        
    q=args.q,
    early_stopping=args.early_stopping,

    # beliefs
    beliefs=args.beliefs,

    # housekeeping
    device=args.device,
    num_trials=args.num_trials,
    num_rounds=args.num_rounds,
    num_epochs=args.num_epochs,
)

In [3]:
ds, ds_dates = plot.generate_data()
net = plot.get_net()

In [90]:
plot.run_trials(ds)

0
0
1
2
3
4
5
6
1
0


KeyboardInterrupt: 